<a href="https://colab.research.google.com/github/B3ARBU/AI-Green-Screen/blob/main/AI_Green_Screen_Video_Convenabil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Edit > Notebook settings / Runtime > Change runtime type > GPU**

CTRL + F9 pentru a începe şi când apare Warning > 
RUN ANYWAY


 










# **Introducere**

1.   Conectează-ţi contul de Google Drive. ( Urmează paşii din engleză de mai jos )

In [ ]:
#@title 1.1 Drive Mount
from google.colab import drive 
drive.mount("/content/gdrive")

from IPython.display import clear_output 
clear_output()

In [ ]:
#@title 1.2 Import
from cv2 import cv2 
from PIL import Image

In [ ]:
#@title 1.3 Git Clone
%cd '/content/gdrive/MyDrive'
!git clone https://github.com/B3ARBU/AI-Green-Screen.git

from IPython.display import clear_output 
clear_output()

2.   Descarcă [u2net.pth](https://drive.google.com/file/d/1qIr8f2IFZg_Z5jMKGnjR7DgwZP19B09w/view?usp=sharing) [ [Imagine](https://imgur.com/a/ymiJd6l) ]



( Pasii 1.3 & 2.1 durează cel mai mult dar partea buna e că trebuie activate doar o singură dată după care puteţi trece peste aceşti paşi )


In [ ]:
#@title 2.1 Upload u2net.pth (2nd time cancel)
%cd '/content/gdrive/MyDrive/AI-Green-Screen/saved_models/u2net'
from google.colab import files
uploaded = files.upload()

!rm -R convenabil.txt

from IPython.display import clear_output 
clear_output()


# **Ai Green Screen Video**

3.   Încarcă videoclipul. ( Recomandări: format - mp4 , HD/FHD/4K , 24fps/30fps | durată < 60s | Acestea sunt recomandări, puteţi încărca orcie rezoluţie cu orice frame rate deşi un clip de 10 minute 4k 60fps poate dura o vreme... )

In [ ]:
#@title 3.1 Upload Video
%cd '/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/input'
from google.colab import files
uploaded = files.upload()

import os

dst ='input.mp4' 
os.rename(list(uploaded.keys())[0], dst)

from IPython.display import clear_output 
clear_output()

In [ ]:
#@title 3.2 Create Temp Files
%cd '/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos'
!mkdir input_frames
!mkdir output_frames 
!mkdir u2net_results

from IPython.display import clear_output 
clear_output()

In [ ]:
#@title 3.3 Video Frame Split
#SCHIMBAȚI EXTENSIA FIȘIERULUI DE INTRARE AICI - MOD IMPLICIT: .mp4
video=cv2.VideoCapture('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/input/input.mp4')
fps=video.get(cv2.CAP_PROP_FPS)
count=0
flag=1
while flag:
  flag,image=video.read()
  if flag==0:
    break
  cv2.imwrite('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/input_frames/'+'input'+str(count)+'.png',image)
  count=count+1


In [ ]:
#@title 3.4 U2net Video
%cd '/content/gdrive/MyDrive/AI-Green-Screen'
!python u2net_video.py

from IPython.display import clear_output 
clear_output()


In [ ]:
#@title 3.5 Image Array
img_array=[]

for i in range(count):

  #u2netreusult
  u2netresult=cv2.imread('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/u2net_results/input'+str(i)+'.png')
  #original
  original=cv2.imread('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/input_frames/input'+str(i)+'.png')
  #subimage
  subimage=cv2.subtract(u2netresult,original)
  cv2.imwrite('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/output_frames/output'+str(i)+'.png',subimage)

  #subimage
  subimage=Image.open('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/output_frames/output'+str(i)+'.png')
  #original
  original=Image.open('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/input_frames/input'+str(i)+'.png')

  subimage=subimage.convert("RGBA")
  original=original.convert("RGBA")

  subdata=subimage.getdata()
  ogdata=original.getdata()

  newdata=[]
  for i in range(subdata.size[0]*subdata.size[1]):
    if subdata[i][0]==0 and subdata[i][1]==0 and subdata[i][2]==0:
      newdata.append((50,255,50,255))
    else:
      newdata.append(ogdata[i])
  subimage.putdata(newdata)
  subimage.save('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/output_frames/output'+str(i)+'.png',"PNG")

  final_img=cv2.imread('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/output_frames/output'+str(i)+'.png')
  ht,wd,l=final_img.shape
  sz=(wd,ht)
  img_array.append(final_img)


In [ ]:
#@title 3.6 Compile
#SCHIMBAȚI EXTENSIA FIȘIERULUI DE INTRARE AICI - MOD IMPLICIT: .mp4
outv=cv2.VideoWriter('/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/output/output.mp4',cv2.VideoWriter_fourcc(*'MP4V'),fps,sz)

for i in img_array:
  outv.write(i)

outv.release()


In [ ]:
#@title 3.7 Delete Temp Files
%cd '/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos'
!rm -R u2net_results
!rm -R input_frames
!rm -R output_frames

from IPython.display import clear_output 
clear_output()

In [ ]:
#@title 3.8 Download
%cd '/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/output'
from google.colab import files

files.download('output.mp4')

/content/gdrive/MyDrive/AI-Green-Screen/test_data/videos/output


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>